# ML Reproducibility Challenge 2021

## Imports

In [1]:
import torch
from torch import nn
import json
import argparse
from tqdm import tqdm
import torch.nn.functional as F

## Teachers

In [2]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, is_last=False):
        super(BasicBlock, self).__init__()
        self.is_last = is_last
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        preact = out
        out = F.relu(out)
        if self.is_last:
            return out, preact
        else:
            return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, is_last=False):
        super(Bottleneck, self).__init__()
        self.is_last = is_last
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        preact = out
        out = F.relu(out)
        if self.is_last:
            return out, preact
        else:
            return out

class ResNet(nn.Module):

    def __init__(self, depth, num_filters, block_name='BasicBlock', num_classes=10):
        super(ResNet, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        if block_name.lower() == 'basicblock':
            assert (
                depth - 2) % 6 == 0, 'When use basicblock, depth should be 6n+2, e.g. 20, 32, 44, 56, 110, 1202'
            n = (depth - 2) // 6
            block = BasicBlock
        elif block_name.lower() == 'bottleneck':
            assert (
                depth - 2) % 9 == 0, 'When use bottleneck, depth should be 9n+2, e.g. 20, 29, 47, 56, 110, 1199'
            n = (depth - 2) // 9
            block = Bottleneck
        else:
            raise ValueError('block_name shoule be Basicblock or Bottleneck')

        self.inplanes = num_filters[0]
        self.conv1 = nn.Conv2d(3, num_filters[0], kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(num_filters[0])
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, num_filters[1], n)
        self.layer2 = self._make_layer(block, num_filters[2], n, stride=2)
        self.layer3 = self._make_layer(block, num_filters[3], n, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(num_filters[3] * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        self.to('cuda')

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = list([])
        layers.append(block(self.inplanes, planes, stride,
                      downsample, is_last=(blocks == 1)))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,
                          is_last=(i == blocks-1)))

        return nn.Sequential(*layers)

    def get_feat_modules(self):
        feat_m = nn.ModuleList([])
        feat_m.append(self.conv1)
        feat_m.append(self.bn1)
        feat_m.append(self.relu)
        feat_m.append(self.layer1)
        feat_m.append(self.layer2)
        feat_m.append(self.layer3)
        return feat_m

    def get_bn_before_relu(self):
        if isinstance(self.layer1[0], Bottleneck):
            bn1 = self.layer1[-1].bn3
            bn2 = self.layer2[-1].bn3
            bn3 = self.layer3[-1].bn3
        elif isinstance(self.layer1[0], BasicBlock):
            bn1 = self.layer1[-1].bn2
            bn2 = self.layer2[-1].bn2
            bn3 = self.layer3[-1].bn2
        else:
            raise NotImplementedError('ResNet unknown block error !!!')

        return [bn1, bn2, bn3]

    def forward(self, x, with_features=False, preact=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)  # 32x32
        f0 = x

        x, f1_pre = self.layer1(x)  # 32x32
        f1 = x
        x, f2_pre = self.layer2(x)  # 16x16
        f2 = x
        x, f3_pre = self.layer3(x)  # 8x8
        f3 = x

        x = self.avgpool(x)
        f4 = x
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        if with_features:
            if preact:
                return [f0, f1_pre, f2_pre, f3_pre, f4], x
            else:
                return [f0, f1, f2, f3, f4], x
        else:
            return x

def resnet44(num_classes=10):
    return ResNet(44, [16, 16, 32, 64], 'basicblock', num_classes)

def resnet56(num_classes=10):
    return ResNet(56, [16, 16, 32, 64], 'basicblock', num_classes)

def resnet110(num_classes=10):
    return ResNet(110, [16, 16, 32, 64], 'basicblock', num_classes)

## Students

In [3]:
def resnet8(num_classes=10):
    return ResNet(8, [16, 16, 32, 64], 'basicblock', num_classes)

def resnet14(num_classes=10):
    return ResNet(14, [16, 16, 32, 64], 'basicblock', num_classes)

def resnet20(num_classes=10):
    return ResNet(20, [16, 16, 32, 64], 'basicblock', num_classes)

## Attention Based Fusion

In [4]:
def upsample_and_match_channels(features2, features1, device):
    """Upsamples features2 to match features1"""
    features2 = nn.Conv2d(
        features2.shape[1],
        features1.shape[1],
        kernel_size=1
    ).to(device)(features2)

    return F.interpolate(features2, (features1.shape[2], features1.shape[3]))

def abf(features1, features2, device):
    features2 = upsample_and_match_channels(features2, features1, device)

    features_concat = torch.cat((features1, features2), dim=1)
    n, c, h, w = features_concat.shape

    att_maps = nn.Conv2d(c, 2, kernel_size=1).to(device)(features_concat)
    att_map1, att_map2 = att_maps[:, 0, :, :], att_maps[:, 1, :, :]
    att_map1, att_map2 = att_map1.reshape(
        (n, 1, h, w)), att_map2.reshape((n, 1, h, w))

    return features1 * att_map1 + features2 * att_map2

## Hierarchial Context Loss

In [5]:
def hcl(student_features, teacher_features):
    total_loss = 0.0
    n, c, h, w = student_features.shape

    levels = [h, 4, 2, 1]
    lvl_weight = 1.0
    total_weight = 0.0

    for i in range(n):
        example_loss = 0.0

        for lvl in levels:
            if lvl > h:
                continue

            lvl_sf = F.adaptive_avg_pool2d(student_features, (lvl, lvl))
            lvl_tf = F.adaptive_avg_pool2d(teacher_features, (lvl, lvl))

            lvl_loss = F.mse_loss(lvl_sf, lvl_tf) * lvl_weight
            example_loss += lvl_loss

            total_weight += lvl_weight
            lvl_weight = lvl_weight / 2.0

        total_loss += example_loss / total_weight

    return total_loss

## Hyper-parameters

## Train Loop

In [ ]:
def train(student, teacher, train_iter, loss, optimizer):
    if is_pretrained_present(params, net_type):
        print('using pretrained')
        student = load_model(params, net_type)
        return student

    average_loss = RunningAverage()

    print("\nstarting training")

    student.train()
    teacher.eval()

    for _ in range(num_epochs):
        with tqdm(total=len(train_iter)) as t:
            for X, y in train_iter:
                X, y = X.to(device), y.to(device)

                student_features, student_preds = student(X, with_features=True)
                with torch.no_grad():
                    teacher_features, teacher_preds = teacher(
                        X, with_features=True)

                ce_loss = nn.CrossEntropyLoss()(student_preds, y)

                student_features = student_features[::-1]
                teacher_features = teacher_features[::-1]

                total_kd_loss = 0

                prev_abf_output = student_features[0]

                for sf, tf in zip(student_features[1:], teacher_features[1:]):
                    # print("SF:\n", sf.shape)
                    # print("PREV ABF OUTPUT:\n", prev_abf_output.shape)
                    abf_output = abf(sf, prev_abf_output, device)
                    total_kd_loss += loss(abf_output, tf)
                    prev_abf_output = abf_output
                
                total_loss = ce_loss + total_kd_loss * kd_loss_weight
                # print(total_loss)

                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()

                average_loss.update(total_loss.item())
                t.set_postfix(loss=f'{average_loss():.3f}')
                t.update()

    store_model(student, params, net_type)

    return student